
## 1. 선행지식

### 1.1 IoU(Intersection Over Union)

- Object Detection 문제의 Ground Truth(정답) 데이터는 사람이 지정한 bounding box와 class label임. Ground Truth 데이터와 비교했을 때 예측 값의 성능을 비교하기 위한 측정법이 필요함
- Bounding Box Maching
  - 객체에 대한 정보를 가지는 예측 Bounding box와 정답 Box의 IOU를 구하여 예측값과 실측값(Ground Truth)이 얼마나 겹치는지를 나타내는 지표
  - 한 개의 bounding box와 다른 한 개의 bounding box가 얼마나 일치하는지를 0.0 ~ 1.0 사이의 값으로 표현함
  - 두 개의 bounding box가 일치할수록 1.0에 가까운 값이 되고, 불일치할수록 0.0에 가까운 값이 나오게 됨
  - IoU가 특정 임계값(threshold)보다 높으면 match라고 판정
<div style="text-align:center">
<img src="https://www.pyimagesearch.com/wp-content/uploads/2016/09/iou_stop_sign.jpg" height="50%" width="50%">

<img src="https://pyimagesearch.com/wp-content/uploads/2016/09/iou_examples.png" height="50%" width="50%">

<img src="https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11263-019-01247-4/MediaObjects/11263_2019_1247_Fig20_HTML.png?as=webp" height="50%" width="50%">

</div>

- input: 한 개의 ground-truth(정답) bounding box, 한 개의 predict(예측) box
- 방법
    - 중간점과 높이/너비 대신 왼쪽 위, 오른쪽 아래 두 모서리를 사용하여 상자 정의
    - 직사각형의 면적을 계산하기 위해 높이에 너비를 곱함
    - ground-truth box와 predicted box의 교점을 찾음

    <img src="https://wikidocs.net/images/page/162474/14_CIoU.png">

### 1.2 오차행렬(Confusion Matrix)
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcWtdQm%2FbtrsrRO2ufk%2FfyrgbWNNKk7fZxGjuknkg1%2Fimg.png" width=600 >

- 정밀도(Precision): 예측을 positive로 한 데이터 중 예측값과 실제값이 positive로 일치한 데이터의 비율 (모델이 검출한 T중에 실제 T의 비율)
- 재현율(Recall): 실제로 positive인 데이터 중 예측값과 실제값이 positive로 일치한 데이터의 비율 (전체 T중에 실제로 모델이 검출한 T의 비율)
- `TP(True Positive)`: 실제 True인 정답을 True라고 예측 (정답)
- `FP(False Positive)`: 실제 False인 정답을 True라고 예측 (오답)
- `TN(True Negative)`: 실제 True인 정답을 False라고 예측 (오답)
- `FN(False Negative)`: 실제 False인 정답을 False라고 예측 (정답) 

$\qquad precision = \frac{TP}{TP \ + \ FP}$
$\qquad recall = \frac{TP}{TP \ + \ FN}$

- 정밀도와 재현율은 trade-off 관계임
  - 모델이 안정적이지 않은 특징을 기반으로 객체 존재를 예측하면 FP가 많아져서 정밀도가 낮아짐
  - 모델이 너무 엄격해서 정확한 조건을 만족할 때만 객체가 탐지된 것으로 간주하면 FN이 많아져서 재현율이 낮아짐

<div style="text-align:center">
  <img src="https://velog.velcdn.com/images%2Fsset2323%2Fpost%2F2a3874bc-53b5-4fdd-be09-3f6d91ad721a%2Fimage.png" width=600>
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/525px-Precisionrecall.svg.png" width=25% height=25% >
</div>
    

#### 어떠한 경우에 정밀도와 재현율이 중요할까?
##### 1. 정밀도(스팸 메일)  
  $$ Precision = \frac{TP}{TP \ + \ \colorbox{cornflowerblue}{FP}}$$ 
  - 스팸인 줄 알았는데(Positive) 실제론 아니었다(False) $\rarr$ **False Positive** 
  - 스팸이 아닌 줄 알았는데(Negative) 실제론 스팸이었다(True) $\rarr$ **True Negative**
  - 이러한 경우 `False Positive` 즉, 스팸인 줄 알았는데 실제론 아닌 경우에 잘못하여 중요한 이메일을 스팸으로 거른 경우 penalty가 매우 커짐. 따라서 Positive를 예측할 때 더 신중을 기해야 함
##### 2. 재현율(암 진단)
  $$ Recall = \frac{TP}{TP \ + \ \colorbox{cornflowerblue}{FN}}$$ 
  - 암인 줄 알았는데(Positive) 실제론 아니었다(False) $\rarr$ **False Positive**
  - 암이 아닌 줄 알았는데(Negative) 실제로 암이었다(True) $\rarr$ **True Negative**
  - 이러한 경우 `True Negative` 즉, 암이 아닌 줄 알았는데 실제로 암인 경우에 치료 시기를 놓쳐 결국 사망에 이르게 할 위험이 높아짐. True Negative에 따른 penalty가 매우 크다. 따라서 Negative를 예측할 때 더 신중을 기해야 함.
  


#### 정밀도-재현율 곡선(Precision-Recall Curve) 
  - confidence score 임계값마다 모델의 정밀도와 재현율을 시각화
  - 모든 bounding box와 함께 모델이 예측의 정확성을 얼마나 확실하는지 0 ~ 1사이의 숫자로 나타내는 신뢰도를 출력
  - 임계값(Threshold value) $T$에 따라 정밀도와 재현율이 달라짐
    - 임계값 $T$ 이하의 예측은 제거함
    - $T$가 1에 가까우면 정밀도는 높지만 재현율은 낮음  
      - 놓치는 객체가 많아져서 재현율이 낮아짐. 즉, 신뢰도가 높은 예측만 유지하기때문에 정밀도는 높아짐
    - $T$가 0에 가까우면 정밀도는 낮지만 재현율은 높음  
      - 대부분의 예측을 유지하기때문에 재현율은 높아지고, False Positive가 많아져서 정밀도가 낮아짐
    - Ex1) 모델이 보행자를 탐지하고 있으면 특별한 이유없이 차를 세우더라도 어떤 보행자도 놓치지 않도록 재현율을 높여야 함
    - Ex2)모델이 투자 기회를 탐지하고 있다면 일부 기회를 놓치게 되더라도 잘못된 기회에 돈을 거는 일을 피하기 위해 정밀도를 높여야 함


<div style="text-align:center">
  <img src="https://velog.velcdn.com/images%2Fsset2323%2Fpost%2F2a3874bc-53b5-4fdd-be09-3f6d91ad721a%2Fimage.png" height="50%" width="50%">
  
  <img src="https://github.com/dariyasydykova/open_projects/raw/master/ROC_animation/animations/PR.gif">
</div>



### 1.3 mAP(Mean Average Precision)

- AP(Average Precision)
  - 알고리즘의 성능을 하나의 값으로 표현한 것으로써, Precision-Recall Curve(이하 PR Curve)에서 그래프 선 아래 쪽의 면적으로 계산된다.
  - Average precision이 높으면 높을수록 그 알고리즘의 성능이 전체적으로 우수하다는 의미이다.
  <div style="text-align:center">
    <img src="./img/mAP0.png" width=800><br>
    <img src="./img/mAP1.png" width=800>
  </div>

  
<img src="https://cdn-images-1.medium.com/max/1600/1*9ordwhXD68cKCGzuJaH2Rg.png" width=600>

<img src="https://cdn-images-1.medium.com/max/1600/1*pmSxeb4EfdGnzT6Xa68GEQ.jpeg" width=600>

- mAP(Mean Average Precision)
  - 객체 class가 여러개인 경우 각 클래스별로 Average Precision 값이 나올 것이고, 이것을 모두 더한 뒤 클래스 수로 나눠 얻는 값을 mean Average Precision(mAP)라고 한다.  

<img src="https://cdn-images-1.medium.com/max/1600/1*dEfFSY6vFPSun96lRoxOEw.jpeg" width=600>



### 1.4 NMS(Non-Maximum Suppression)

- 하나의 instasnce에 여러개의 예측 box가 생성될 수 있음
- 각 그리드 셀(grid cell)에 대한 각 클래스별 확률 값을 계산하여 예측된 여러 개의 bounding box에서 최대 확률을 가진 bounding box를 한 개 선택하기 위해 NMS 알고리즘이 사용됨
- 개체에 가장 적합한 경계 상자를 선택하고 다른 모든 경계상자를 거부하거나 억제하는 것을 목적으로 함

<img src="https://naknaklee.github.io/public/images/2021-03-08-NMS-1.png" width=600>

- input
  - B: proposal box 리스트
  - S: confidence 점수
  - N: overlap threshold
- output
    - D: 최종 proposal 리스트
- Process  
  step 1) S가 가장 높은 proposal을 선택하여 B에서 제거 후 D에 추가함 (맨 처음 D는 빈 상태)  
  step 2) 해당 proposal의 IoU을 다른 모든 proposal의 IoU와 비교하여 IoU가 N보다 큰 경우 B에서 해당 proposal을 제거함  
  step 3) B의 나머지 proposal 중 S가 높은 proposal을 B에서 제거 후 D에 추가  
  step 4) 선택한 proposal의 IoU와 다른 모든 proposal의 IoU를 비교하여 N보다 높은 box를  B에서 제거  
  step 5) B에 더 이상 proposal이 남지 않을 때까지 스텝을 반복  
  step 6) 각각의 class에 대해 위 과정을 반복

<img src="https://wikidocs.net/images/page/163295/NMS_8.png">

<img src="https://naknaklee.github.io/public/images/2021-03-08-NMS-2.png" width=500>


# IOU 실습
이전 교재에서 배웠듯 IOU값을 계산하는 공식은 아래와 같습니다.
$$IOU = \frac{Intersection}{A+B-Intersection}$$
위 공식을 활용하여 IOU값을 계산하는 코드를 완성해봅시다!


In [1]:
# function IOU : 두 개의 box를 입력받아 IOU 값을 return 하는 함수
# parameter box1 : predicted(예측) box의 좌표 list (x1, y1, x2, y2)
# parameter box2 : ground-truth(정답) Bounding box의 좌표 list (x1, y1, x2, y2)
def IOU(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    
    # Stage 1: 교차 영역(intersection area)의 높이에 너비를 곱하는 코드를 작성해보세요!
   
    
    inter_area = max(yi2 - yi1, 0) * max(xi2 - xi1, 0)
    
    


    box1_area = (box1[3] - box1[1])*(box1[2]- box1[0])
    box2_area = (box2[3] - box2[1])*(box2[2]- box2[0])
    
    # Formula: Union(A,B) = A + B - Inter(A,B)
    # Stage 2: 다음의 공식을 참고하여 union_area를 구하는 코드를 작성해보세요!
    
    
    union_area = box1_area + box2_area - inter_area
    
    

    # Stage 3: IoU를 구하는 코드를 작성해보세요!
   
    
    IoU = inter_area / union_area
    
    

    return IoU

### 다음의 예제 입출력을 이용하여 코드를 확인해보세요
- <b>Example Input</b><br>
box1 = [2, 8, 7, 3]<br>
box2 = [3, 7, 9, 1]<br>
IOU(box1, box2)

- <b>Example Output</b><br>
0.8484848484848485

In [2]:
box1 = [2, 8, 7, 3]
box2 = [3, 7, 9, 1]

IOU(box1, box2)

-0.0

# NMS 실습
NMS의 알고리즘과 의사코드(pseudocode)를 참고하여 코드를 완성해봅시다!
- Step 1. Confidence_Threshold 및 IOU_Threshold에 대한 값을 정의합니다.
- Step 2. 경계 상자를 신뢰도가 높은 내림차순으로 정렬합니다.
- Step 3. 신뢰도가 < Confidence_Threshold인 상자를 제거합니다.
- Step 4. 신뢰도가 가장 높은 상자부터 시작하여 나머지 모든 상자를 반복합니다.
- Step 5. 동일한 클래스에 속하는 나머지 모든 상자를 사용하여 현재 상자의 IOU를 계산합니다.
- Step 6. 2개 상자의 IOU > IOU_Threshold인 경우 상자 목록에서 신뢰도가 낮은 상자를 제거합니다.
- Step 7. 목록의 모든 상자를 살펴볼 때까지 이 작업을 반복합니다.
 <br><br>
<img src="https://naknaklee.github.io/public/images/2021-03-08-NMS-2.png" style="width:600px">

In [9]:
def nms(boxes, conf_threshold=0.7, iou_threshold=0.4):
    
	bbox_list_thresholded = []	# 낮은 신뢰도의 상자를 필터링한 후의 새 상자 목록을 포함하는 리스트
	bbox_list_new = []			# NMS 수행 후 최종 상자 목록을 포함하는 리스트 
	# Stage 1: 신뢰도 < Confidence_Threshold인 상자를 제거하는 코드를 작성해보세요!     
	boxes_sorted = sorted(boxes, reverse=True, key = lambda x : x[5])	# box들을 내림차순으로 정렬
	for box in boxes_sorted:
		if box[5] > conf_threshold:		
			bbox_list_thresholded.append(box)	
		else:
			pass
	# Stage 2: 모든 정렬된 상자를 반복하여, conf_threshold 보다 낮은 신뢰도를 갖는 상자를 제거하는 코드를 작성해보세요!
	while len(bbox_list_thresholded) > 0:
		current_box = bbox_list_thresholded.pop(0)	
		bbox_list_new.append(current_box)			
		for box in bbox_list_thresholded:
			if current_box[4] == box[4]:				
				iou = IOU(current_box[:4], box[:4])		
				if iou > iou_threshold:					
					return bbox_list_thresholded.remove(box)	
	#return bbox_list_new

### 다음의 예제 입출력을 이용하여 코드의 옳고 그름을 확인해보세요
- <b>Example Input</b><br>
bbox_dog1 = [90, 261, 228, 378, "Dog", 0.9]		&nbsp;&nbsp;&nbsp; # Defining the different bounding boxes<br>
bbox_dog2 = [121, 290, 216, 374, "Dog", 0.6]	<br>
bbox_dog3 = [49, 265, 243, 388, "Dog", 0.85]	<br>
bbox_person1 = [234, 91, 359, 370, "Person", 0.95]	<br>
bbox_person2 = [239, 116, 359, 374, "Person", 0.45]	<br>
bbox_person3 = [234, 71, 359, 370, "Person", 0.92]	<br>
<br>
bbox_list = [bbox_dog1, bbox_dog2, bbox_dog3, bbox_person1, bbox_person2, bbox_person3]	&nbsp;&nbsp;&nbsp;# Creating the list of the boxes <br>
<br>
bbox_list_new = nms(bbox_list, conf_threshold=0.7, iou_threshold=0.4)	&nbsp;&nbsp;&nbsp;# Call the function to perform NMS <br>
<br>
print(bbox_list_new) <br><br>

- <b>Example Output</b><br>
[[234, 91, 359, 370, 'Person', 0.95], [90, 261, 228, 378, 'Dog', 0.9]]

In [7]:
bbox_dog1 = [90, 261, 228, 378, "Dog", 0.9]
bbox_dog2 = [121, 290, 216, 374, "Dog", 0.6]
bbox_dog3 = [49, 265, 243, 388, "Dog", 0.85]
bbox_person1 = [234, 91, 359, 370, "Person", 0.95]
bbox_person2 = [239, 116, 359, 374, "Person", 0.45]	
bbox_person3 = [234, 71, 359, 370, "Person", 0.92]

bbox_list = [bbox_dog1, bbox_dog2, bbox_dog3, bbox_person1, bbox_person2, bbox_person3]

bbox_list_new = nms(bbox_list, conf_threshold=0.7, iou_threshold=0.4)
print(bbox_list_new) 

[[234, 91, 359, 370, 'Person', 0.95], [90, 261, 228, 378, 'Dog', 0.9]]


## YOLOv3 모델 (PyTorch)
이번에는 github(https://github.com/ultralytics/yolov3)에서 yolo3 모델을 가져와 간단한 추론을 해보는 실습을 진행해보겠습니다

### 모델 다운로드
